In [ ]:
import os
os._exit(0)

In [ ]:
import os
os.environ['HF_HOME'] = '/home/stinky/Documents/semantic-robot/ros2_ws/src/mistral_interface/model'

import torch
torch.cuda.empty_cache()

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = 'cuda'
model_id = 'mistralai/Mistral-7B-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
model = AutoModelForCausalLM.from_pretrained('mistralai/Mistral-7B-v0.1', device_map='auto', load_in_4bit=True)

In [ ]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

In [ ]:
commands = [
    "Drive forward 1 meter",
    "Drive backward 4 meters",
    "Turn 45 degrees to the left",
    "Turn 13 degrees to the right",
    "Drive backward 7 meters",
    "Drive forward 2 meters",
    "I like to eat ice cream",
    "Now I think it's time for you to drive forward 5 meters",
    "After much thought, I have decided that you should turn 87 degrees to the left",
    "The pyramids in Egypt were built a long time ago",
    "Eight pots of gold are enough to buy a new car",
    "Yep, I agree that you should drive backward 3 meters",
    "Ugh, now I've had enough of driving forward 1 meter. Drive backward 2 meters",
    "No, don't drive backward. Turn 30 degrees to the right first",
    "Okay, now it's time for you to drive forward 4 meters",
    "Just turn 56 degrees to the right",
    "If you can jump on a pogo stick, you can jump on a pogo stick. Turn 4 degrees to the left",
    "Drive backward 8 meters. Whistle a tune while you do it",
    "Back to the start. Drive forward 1 meter",
    "Turn to the right"
]

In [ ]:
for command in commands:
    messages = [
        {'role': 'user', 'content': 'You are a differential drive robot called Turtle. You can drive forward, backward, rotate left and rotate right. You receive instructions where you will drive. You can only answer in the following format: \"verb|magnitude|unit\". You task: process instructions, output correct verb, magnitude and unit in correct format. If you cannot find a response, answer \"error\" and never ever anything else. To rotate, left/right to correct angle. To drive, forward/backward for x meters.'},
        {'role': 'assistant', 'content': 'My name is Turtle and I am a differential drive robot.'},
        {'role': 'user', 'content': 'Drive forward for 14 meters.'},
        {'role': 'assistant', 'content': 'forward|14|m'},
        {'role': 'user', 'content': 'Rotate left for 90 degrees.'},
        {'role': 'assistant', 'content': 'left|90|d'},
        {'role': 'user', 'content': 'I want to buy a new car. Check how much money I have in my bank account.'},
        {'role': 'assistant', 'content': 'error'},
        {'role': 'user', 'content': command}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors='pt')

    model_inputs = encodeds.to(device)
    # model.to(device)

    generated_ids = model.generate(model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=100, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)

    # Remove the pre-prompts and end-of-sentence token
    output_tokens = decoded[0]
    end_token = "[/INST]"

    # Find last occurence of end_tag in output
    end_tag_index = output_tokens.rfind(end_token)
    end_of_sentence = -4
    sliced_output = output_tokens[end_tag_index + len(end_token):end_of_sentence]
    
    torch.cuda.empty_cache()

    print(sliced_output)
    
print(model.get_memory_footprint())